<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Red%20convolucional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ARRANCAMOS CON LO BÁSICO
import numpy as np
import tensorflow as tf
import pandas as pd
from google.colab import files

!pip install -q tensorflow-hub
import tensorflow_hub as hub

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


Version:  2.2.0
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [2]:
# CARGAMOS LOS CSV
url_train = 'https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'
url_test = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test.csv'

train_tmp_df = pd.read_csv(url_train)
test_df = pd.read_csv(url_test)
train_tmp_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
# LIMPIAMOS Y CHUSMEAMOS
train_tmp_df = train_tmp_df.drop(['keyword', 'location'], axis = 1)
train_tmp_df.head()
train_tmp_df.shape

(7613, 3)

In [13]:
# SUSPENDIDO: EXTRAEMOS UN SET DE VALIDACIÓN DEL 10%
#from sklearn.model_selection import train_test_split

#train_df, validation_df = train_test_split(train_tmp_df, test_size = 0.1)
#train_df.shape[0]


#TEMPORARIO:
train_df = train_tmp_df

In [5]:
# SUSPENDIDO:
#validation_df.shape[0]

762

In [14]:
# HACEMOS UN EMBEDDING PROVISTO POR UN "TensorFlow Hub model"

embedding_url = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding_url,
                           input_shape = [], 
                           dtype = tf.string,
                           trainable = True)

train_text_list = np.asarray(train_df['text'])
train_target_list = np.asarray(train_df['target'])

hub_layer(train_text_list[:3])



<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 0.00723878,  0.15689617,  1.2953067 , -0.04077986, -0.16036902,
        -1.2004429 , -0.40451345, -0.5239726 , -0.23558664, -0.35022184,
        -0.28988177, -0.47197863, -0.07621863,  0.17048942, -0.8323773 ,
         0.02260084,  0.16125458, -0.64086777, -0.569074  ,  0.04000134],
       [-0.4983734 ,  1.2828879 , -0.18836124, -0.94734734,  0.8805808 ,
        -0.17885518, -0.01543166,  0.3832704 , -1.508848  ,  1.7532811 ,
        -0.75493276,  0.60124093, -0.14378418, -0.891944  ,  0.43213663,
         1.3467408 , -1.4859253 ,  1.5762157 , -0.37878054,  0.9423684 ],
       [ 0.08814443, -0.31140247,  0.6044149 ,  1.7130805 ,  1.3203121 ,
        -3.201303  ,  0.8445131 ,  0.9621795 , -1.5353138 ,  2.2069528 ,
        -0.2790253 ,  0.01095965, -2.2282734 ,  0.2043116 , -2.17501   ,
         1.3441613 ,  0.11884003, -1.2388006 , -0.25501075,  0.69527215]],
      dtype=float32)>

In [15]:
# ARMAMOS EL MODELO
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_3 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_4 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 400,645
Trainable params: 400,645
Non-trainable params: 0
_________________________________________________________________


In [17]:
# COMPILAMOS Y ENTRENAMOS

# NOTAS:
# Con 20 epochs obtiene mejor resultado que con 30

model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'])

history = model.fit(train_text_list, train_target_list, batch_size=512,
                    epochs = 20,
                    verbose = 1)

In [19]:
# SUSPENDIDO: EVALUAMOS CON EL SET DE VALIDACION
#validation_text_list = np.asarray(train_df['text'])
#validation_target_list = np.asarray(train_df['target'])

#loss, precision = model.evaluate(validation_text_list, validation_target_list)
#print('Precisión: %.2f' % (precision*100))

#TEMPORARIO:
loss, precision = model.evaluate(train_text_list, train_target_list)
print('Precisión: %.2f' % (precision*100))


238/238 [==============================] - 0s 2ms/step - loss: 0.1391 - accuracy: 0.9417
Precisión: 94.17


In [20]:
# ARMAMOS LA PREDICCIÓN
test_text_list = np.asarray(test_df['text'])
prediction = (model.predict(test_text_list) > 0.5).astype("int32")
prediction

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [0]], dtype=int32)

In [21]:
# EXPORTAMOS EL CSV EN EL FORMATO QUE PIDE KAGGLE
test_df['target'] = prediction
export = pd.DataFrame()
export['id'] = test_df['id']
export['target'] = test_df['target']
export = export.set_index('id')
export.to_csv('submit.csv')
files.download('submit.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>